In [4]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [30]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, Add, Activation, Dropout

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1


2024-01-12 12:03:33.813370: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 12:03:40.624421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38420 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


tf.Tensor(-1109.957, shape=(), dtype=float32)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [25]:
from tensorflow.keras.layers import Conv3D, Add, ReLU, BatchNormalization
from tensorflow.keras import Input, Model

def conv3d_block(x, filters):
    conv_1 = Conv3D(filters, 4, padding='same')(x)
    norm_1 = BatchNormalization()(conv_1)
    relu_1 = ReLU()(norm_1)
    conv_2 = Conv3D(filters, 4, padding='same')(relu_1)
    norm_2 = BatchNormalization()(conv_2)
    return norm_2

def identity_block(tensor, filters):
    x = conv3d_block(tensor, filters)
    output = Add()([x, tensor])
    output = ReLU()(output)
    return output

def projection_block(tensor, filters):
    x = conv3d_block(tensor, filters)
    projection = Conv3D(filters, 1, padding='same')(tensor)
    output = Add()([x, projection])
    output = ReLU()(output)
    return output

def resnet_block(x, filters, downsample=False):
    if downsample:
        x = Conv3D(filters, 1, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        
    y = conv3d_block(x, filters)
    y = Add()([y, x])
    y = ReLU()(y)
    return y


In [27]:
def create_model(input_shape):
    input_layer = Input(input_shape)
    # repating ResNet block
    x = resnet_block(input_layer, 32)
    x = resnet_block(x, 64, downsample=True)
    
    # Down sampling b4 next block
    x = Conv3D(128, 1, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = resnet_block(x, 128)
    
    # Down next block
    x = Conv3D(256, 1, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = resnet_block(x, 256)
    
    # Final layers leading to output
    x = Conv3D(128, 3, activation="relu", padding="same")(x)
    x = UpSampling3D((2, 2, 2))(x)
    x = Conv3D(64, 3, activation="relu", padding="same")(x)
    x = UpSampling3D((2, 2, 2))(x)
    output = Conv3D(1, (3, 3, 3), activation='linear', padding='same')(x)
   
    model = Model(inputs=[input_layer], outputs=[output])
    return model

In [7]:
data = np.load('./generated_data/simulated_data_010124_5K.npy')

In [8]:
os.environ['PYTHONHASHSEED']=str(123)

np.random.seed(123)
tf.random.set_seed(123)

In [9]:
# Split the data
initial_state = data[:, 0]
final_state = data[:, 1]

# Compute the mean and std of the initial and final states
initial_state_mean, initial_state_std = np.mean(initial_state), np.std(initial_state)
final_state_mean, final_state_std = np.mean(final_state), np.std(final_state)

# Normalize the initial and final states
initial_state = (initial_state - initial_state_mean) / initial_state_std
final_state = (final_state - final_state_mean) / final_state_std

# Select one simulation's initial and final states
initial_conditions = initial_state[0]
final_density = final_state[0]

In [10]:
# train and test
X_train, X_test, y_train, y_test  = train_test_split(initial_state, final_state, test_size=0.15, random_state=123)

# further splitting it in training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=123)

In [11]:
# Recasting dimensions to 4D tensor (suitable for 3D convolutions) with the following dimensions: (batch_size, height, width, depth, num_channels)
X_train = np.reshape(X_train, (*X_train.shape, 1))
X_val = np.reshape(X_val, (*X_val.shape, 1))
X_test = np.reshape(X_test, (*X_test.shape, 1))

y_train = np.reshape(y_train, (*y_train.shape, 1))
y_val = np.reshape(y_val, (*y_val.shape, 1))
y_test = np.reshape(y_test, (*y_test.shape, 1))

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((3612, 32, 32, 32, 1),
 (3612, 32, 32, 32, 1),
 (638, 32, 32, 32, 1),
 (638, 32, 32, 32, 1),
 (750, 32, 32, 32, 1),
 (750, 32, 32, 32, 1))

In [38]:
# Parameters
Ngrid = 32  # grid size

In [39]:
res_model = create_model((Ngrid, Ngrid, Ngrid, 1))

ValueError: Inputs have incompatible shapes. Received shapes (32, 32, 32, 128) and (32, 32, 32, 64)

In [32]:
# Compile the model
res_model.compile(optimizer=Adam(), 
                   loss='mse',
                   metrics=['mae'],
                   run_eagerly=True)

In [33]:
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=5, 
                              min_lr=0.001)
checkpoint = ModelCheckpoint('resnet_5K.hdf5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True)

In [34]:
history = res_model.fit(x=X_train, 
                         y=y_train, 
                         validation_data=(X_val, y_val), 
                         batch_size=32, 
                         epochs=50, 
                         callbacks=[early_stopping, reduce_lr, checkpoint])

Epoch 1/50


2024-01-12 12:23:01.584561: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: required broadcastable shapes [Op:SquaredDifference]

In [ ]:
res_model.summary()